<a href="https://colab.research.google.com/github/ChoSooBeen/DeepLearning_with_PyTorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN 실습

In [1]:
import torch
import torch.nn as nn

In [2]:
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(32, 3, 256, 256)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([32, 3, 256, 256])


### 합성곱층과 풀링 선언

In [3]:
conv1 = nn.Conv2d(3, 32, 3, padding=1)
print(conv1)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


### 입력된 텐서 크기 변화

In [6]:
out = conv1(inputs)
print(out.shape)

torch.Size([32, 32, 256, 256])


In [7]:
out = pool(out)
print(out.shape)

torch.Size([32, 32, 128, 128])


In [8]:
out = conv2(out)
print(out.shape)

torch.Size([32, 64, 128, 128])


In [9]:
out = pool(out)
print(out.shape)

torch.Size([32, 64, 64, 64])


In [10]:
out.size(0)

32

In [11]:
out.size(1)

64

In [12]:
out.size(2)

64

In [13]:
out.size(3)

64

In [14]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1) 
print(out.shape)

torch.Size([32, 262144])


In [15]:
fc = nn.Linear(262144, 309) # input_dim = 262144, output_dim = 309
out = fc(out)
print(out.shape)

torch.Size([32, 309])


# CNN으로 이미지 분류해보기

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [18]:
learning_rate = 0.001
training_epochs = 15
batch_size = 32

### 이미지 가져오기

In [19]:
import zipfile
import gdown

In [20]:
google_path = 'https://drive.google.com/uc?id='
file_id = '1PwuXmL0HdmjsThBHDKhaCfKK3jvdoAIG'
output_name = 'train.zip'
gdown.download(google_path+file_id,output_name)

Downloading...
From: https://drive.google.com/uc?id=1PwuXmL0HdmjsThBHDKhaCfKK3jvdoAIG
To: /content/train.zip
100%|██████████| 1.37G/1.37G [00:12<00:00, 113MB/s]


'train.zip'

In [21]:
input_path = './train.zip'
output_path = './data'

In [22]:
zip_data = zipfile.ZipFile(input_path)
zip_data.extractall(output_path)

In [23]:
train_path = './data/output2/train'
val_path = './data/output2/val'
test_path = './data/output2/test'

### Dataset 정의

In [24]:
import torchvision
from torchvision import transforms

In [25]:
trainset = torchvision.datasets.ImageFolder(root = train_path, transform=transforms.ToTensor())
valset = torchvision.datasets.ImageFolder(root = val_path, transform=transforms.ToTensor())
testset = torchvision.datasets.ImageFolder(root = test_path, transform=transforms.ToTensor())

In [27]:
data_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [28]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 256, 256, 3)
        #    Conv     -> (?, 256, 256, 32)
        #    Pool     -> (?, 128, 128, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 128, 128, 32)
        #    Conv      ->(?, 128, 128, 64)
        #    Pool      ->(?, 64, 64, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 64x64x64 inputs -> 309 outputs
        self.fc = torch.nn.Linear(64 * 64 * 64, 309, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [29]:
# CNN 모델 정의
model = CNN().to(device)

In [30]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 1076


In [42]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 3.06402445
[Epoch:    2] cost = 0.44649142
[Epoch:    3] cost = 0.210068017
[Epoch:    4] cost = 0.148407832
[Epoch:    5] cost = 0.104064554
[Epoch:    6] cost = 0.0660645366
[Epoch:    7] cost = 0.0591171235
[Epoch:    8] cost = 0.0470294803
[Epoch:    9] cost = 0.0339070186
[Epoch:   10] cost = 0.025868848
[Epoch:   11] cost = 0.0196676832
[Epoch:   12] cost = 0.0297301132
[Epoch:   13] cost = 0.0187802725
[Epoch:   14] cost = 0.010671421
[Epoch:   15] cost = 0.016225297


In [52]:
val_loader = torch.utils.data.DataLoader(dataset=valset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [57]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        correct_prediction = torch.argmax(outputs, 1) == y
        accuracy = correct_prediction.float().mean()
        print(f"Accuracy: {accuracy.item()}")
        print("--------------------")

Accuracy: 0.9375
--------------------
Accuracy: 0.9375
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.90625
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.9375
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.90625
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.90625
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 1.0
--------------------
Accuracy: 1.0
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.875
--------------------
Accuracy: 0.90625
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.96875
-----